## Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [2]:
def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    solved_sudoku = sudoku.copy()
        
    empty_cells = get_empty_cells(solved_sudoku)    
    solution_values = {}
    
    for cell in empty_cells:
        solution_values[cell] = -1
    
    solution = solve(solved_sudoku, empty_cells, solution_values)

    if solution is not None:
        if validate_result(solved_sudoku) == False:
            solved_sudoku = [[-1]*9]*9
    else:
        solved_sudoku = [[-1]*9]*9
        
    return solved_sudoku

def get_empty_cells(sudoku):
    empty_cells = []
    for row in range(len(sudoku)):
        for col in range(len(sudoku[0])):
            if sudoku[row, col] == 0:
                empty_cells.append((row,col))
    return empty_cells
    
def is_empty(sudoku):
    for row in range(len(sudoku)):
        for col in range(len(sudoku[0])):
            if sudoku[row, col] == 0:
                return (row, col)
            return False
            
def is_valid_number(sudoku, number, cell):
    """
    Checks if given value in an empty cell validates against sudoku constraints.
    """
    
    # check if the number already exists in a row
    for row in range(len(sudoku)):
        if sudoku[cell[0]][row] == number and row != cell[1]:
            return False
    #check if the number already exists in a column
    for col in range(len(sudoku)):
        if sudoku[col][cell[1]] == number and col != cell[0]:
            return False
    
    #small box row
    sbr = cell[0]//3
    #small box cell
    sbc = cell[1]//3

    # check is the number already exists in a 3x3 section
    for row in range(0, len(sudoku)//3):
        for col in range(0, len(sudoku)//3):
            if sudoku[(sbr*3)+row][(sbc*3)+col] == number and ((sbr*3)+row,(sbc*3)+col) != cell:
                return False
    return number

def solve(sudoku, empty_cells, solution_values):
    """
    Solves sudoku using constraint satisfaction and recursive backtracking search
    """

    # base case - all cells are solved
    if not -1 in solution_values.values():
        return solution_values
   
    # create a list of cells still to be solved
    not_yet_solved = []
    
    for cell in empty_cells:
        if solution_values[cell] == -1:
            not_yet_solved.append(cell)
            
    # get possible solutions for a single cell
    cell_domain = get_cell_domain(sudoku, not_yet_solved[0])
    
    single_cell = not_yet_solved[0]
    for val in cell_domain:
        solution_values_copy = solution_values.copy()
        solution_values_copy[single_cell] = val
        for key, value in solution_values_copy.items():
            sudoku[key[0]][key[1]] = value
        solution = solve(sudoku, empty_cells, solution_values_copy)
        if solution is not None:
            return solution
    
    return None

def get_cell_domain(sudoku, cell):
    """
    Creates all possible values for a given cell
    """
    cell_domain = []
    for number in range(1,10):
        if is_valid_number(sudoku, number, cell):
            cell_domain.append(number)
    return cell_domain
    
def validate_result(sudoku):
    """
    Validates final results
    """
    for row in range(len(sudoku)):
        for col in range(len(sudoku[0])):
            if is_valid_number(sudoku, sudoku[row, col], (row, col)) == False:
                return False
    return True

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.


In [3]:

import time
difficulties = ['very_easy', 'easy', 'medium', 'hard']
#     difficulties = ['medium', 'hard']

for difficulty in difficulties:
    print(f"Testing {difficulty} sudokus")
        
    sudokus = np.load(f"data/{difficulty}_puzzle.npy")
    solutions = np.load(f"data/{difficulty}_solution.npy")
        
    count = 0
    for i in range(len(sudokus)):
        sudoku = sudokus[i].copy()
        print(f"This is {difficulty} sudoku number", i)
        print(sudoku)
            
        start_time = time.process_time()
        your_solution = sudoku_solver(sudoku)
        end_time = time.process_time()
            
        print(f"This is your solution for {difficulty} sudoku number", i)
        print(your_solution)
            
        print("Is your solution correct?")
        if np.array_equal(your_solution, solutions[i]):
            print("Yes! Correct solution.")
            count += 1
        else:
            print("No, the correct solution is:")
            print(solutions[i])
            
        print("This sudoku took", end_time-start_time, "seconds to solve.\n")

    print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
    if count < len(sudokus):
        break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.00276090700000009 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 

This is your solution for hard sudoku number 0
[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 6.054727817 seconds to solve.

This is hard sudoku number 1
[[1 0 0 7 0 0 0 0 0]
 [0 3 2 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0]
 [0 8 0 0 0 2 0 7 0]
 [5 0 7 0 0 1 0 0 0]
 [0 0 0 0 0 3 6 1 0]
 [7 0 0 0 0 0 2 0 9]
 [0 0 0 0 5 0 0 0 0]
 [3 0 0 0 0 4 0 0 5]]
This is your solution for hard sudoku number 1
[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -

This is your solution for hard sudoku number 14
[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 48.223444325 seconds to solve.

15/15 hard sudokus correct
